In [1]:
import pandas as pd
import numpy as np
from database import database
import math

import statistics
import datetime

In [2]:
chart_calc = []
for calc in database.calculation_collection.find({}):
    chart_calc.append(calc)

In [3]:
chart_calc_df = pd.DataFrame(chart_calc)

In [4]:
chart_calc_df[chart_calc_df["counterparty"] == 'AAPL']

,_id,counterparty,date,average_score,sentiments,keyword_count,news_count
12,617a688cac1b9752c3bec92b,AAPL,2021-03-08,0.142857,"{'1': 4, '-1': 2, '0': 8, 'rolling_avg': 0.085...","{'fine': 1, 'suit': 1, 'acquisition': 1}",14
20,617a688cac1b9752c3bec930,AAPL,2020-11-05,0.142857,"{'-1': 17, '0': 61, '1': 35, 'rolling_avg': 0....","{'sanction': 1, 'fail': 2, 'late': 12, 'accele...",113
21,617a688cac1b9752c3bec92d,AAPL,2021-08-22,0.142857,"{'1': 2, '0': 12, 'rolling_avg': 0.16607509046...",{},14
23,617a688cac1b9752c3bec933,AAPL,2021-05-11,0.022222,"{'0': 60, '-1': 14, '1': 16, 'rolling_avg': 0....","{'fraud': 1, 'accelerate': 2, 'loss': 4, 'fail...",90
27,617a688cac1b9752c3bec93f,AAPL,2021-04-04,0.000000,"{'0': 1, 'rolling_avg': 0.2677610558556065, 'r...",{},1
...,...,...,...,...,...,...,...
23001,622530360f24882f46ff4e3d,AAPL,2022-03-06,NaN,"{'0': 8, '-1': 1, 'rolling_avg': 0.06265401221...",{'suspend': 1},9
23024,62268353c475d78a84850b1e,AAPL,2022-03-07,NaN,"{'-1': 8, '0': 47, '1': 12, 'rolling_avg': 0.0...","{'late': 6, 'sanction': 2, 'suspend': 1}",67
23147,6227d7d4c475d78a84f99fe2,AAPL,2022-03-08,NaN,"{'-1': 2, '0': 82, '1': 30, 'rolling_avg': 0.2...","{'late': 5, 'loss': 2, 'sanction': 5, 'losses'...",114
23194,62292789c475d78a84718dc1,AAPL,2022-03-09,NaN,"{'-1': 7, '0': 30, '1': 28, 'rolling_avg': 0.3...","{'sanction': 4, 'support': 1, 'loss': 5, 'fail...",65


In [5]:
## what is the plan?
## look at the last few data points and see if something weird has happened: have to define a threshold
## look at average score

In [6]:
chart_calc_df[chart_calc_df["counterparty"] == 'AAPL']["sentiments"]

12       {'1': 4, '-1': 2, '0': 8, 'rolling_avg': 0.085...
20       {'-1': 17, '0': 61, '1': 35, 'rolling_avg': 0....
21       {'1': 2, '0': 12, 'rolling_avg': 0.16607509046...
23       {'0': 60, '-1': 14, '1': 16, 'rolling_avg': 0....
27       {'0': 1, 'rolling_avg': 0.2677610558556065, 'r...
                               ...                        
23001    {'0': 8, '-1': 1, 'rolling_avg': 0.06265401221...
23024    {'-1': 8, '0': 47, '1': 12, 'rolling_avg': 0.0...
23147    {'-1': 2, '0': 82, '1': 30, 'rolling_avg': 0.2...
23194    {'-1': 7, '0': 30, '1': 28, 'rolling_avg': 0.3...
23236    {'0': 27, '1': 11, '-1': 8, 'rolling_avg': 0.2...
Name: sentiments, Length: 508, dtype: object

In [7]:
chart_calc_df.sort_values(by='date', ascending=False)

,_id,counterparty,date,average_score,sentiments,keyword_count,news_count
23270,622a7725c475d78a84e12db5,BSAQ,2022-03-10,NaN,"{'0': 13, '1': 4, '-1': 3, 'rolling_avg': 0.19...","{'loss': 1, 'late': 4}",20
23220,622a00c3c475d78a84bb74a6,CVS,2022-03-10,NaN,"{'0': 6, '1': 2, 'rolling_avg': 0.558966976734...",NaN,8
23238,622a00c5c475d78a84bb8338,LCID,2022-03-10,NaN,"{'0': 3, '1': 3, '-1': 1, 'rolling_avg': 0.100...",{'late': 1},7
23237,622a00c5c475d78a84bb82f6,TGT,2022-03-10,NaN,"{'1': 5, '0': 3, 'rolling_avg': 0.537658003410...",NaN,8
23236,622a00c5c475d78a84bb8183,AAPL,2022-03-10,NaN,"{'0': 27, '1': 11, '-1': 8, 'rolling_avg': 0.2...","{'suspend': 1, 'late': 6, 'acquisition': 2}",46
...,...,...,...,...,...,...,...
17582,61e01962e960cc6ac8e47f06,TXN,2020-03-24,NaN,"{'0': 1, 'rolling_avg': 0.0, 'rolling_weight':...",{},1
1468,617a688eac1b9752c3bed6a0,AAPL,2020-03-18,0.0,"{'0': 1, 'rolling_avg': 0.0, 'rolling_weight':...",{},1
13065,61e01960e960cc6ac8e467f8,TSLA,2018-04-05,NaN,"{'0': 1, 'rolling_avg': 0.0, 'rolling_weight':...",{},1
11588,61e0195fe960cc6ac8e46003,MS,2018-04-05,NaN,"{'0': 1, 'rolling_avg': 0.0, 'rolling_weight':...",{},1


In [ ]:
def mean(data):
    n = len(data)
    mean = sum(data) / n
    return mean
 
def variance(data):
    n = len(data)
    mean = sum(data) / n
    deviations = [(x - mean) ** 2 for x in data]
    variance = sum(deviations) / n
    return variance
 
def stdev(data):
    var = variance(data)
    std_dev = math.sqrt(var)
    return std_dev
 
print("Mean of the sample is % s " % (mean(data))) 
print("Standard Deviation of the sample is % s "% (stdev(data)))

In [94]:
def rolling_avg_collector(counterparty: str):
    counterparty_df = chart_calc_df[chart_calc_df["counterparty"] == counterparty]
    counterparty_df = counterparty_df.sort_values(by='date', ascending=False)

    prev_week_start = datetime.date.today() - datetime.timedelta(days = (datetime.datetime.today().isoweekday() + 14))
    prev_week_stop = datetime.date.today() - datetime.timedelta(days = datetime.datetime.today().isoweekday())
    
    
    roll_avg_arr = []
    negative_sent_arr = []
    positive_sent_arr = []
    
    while (prev_week_start != prev_week_stop):
        date = prev_week_start.strftime("%Y-%m-%d")
        sent_dict = counterparty_df[counterparty_df['date'] == date]['sentiments']
        
        for sent in sent_dict:
            if 'rolling_avg' in sent:
                roll_avg_arr.append(sent['rolling_avg']) 
            else:
                roll_avg_arr.append(0)
                
            if '-1' in sent:
                negative_sent_arr.append(sent['-1'])
            else:
                negative_sent_arr.append(0)
                
            if '1' in sent:
                positive_sent_arr.append(sent['1'])
            else:
                positive_sent_arr.append(0)
        
        prev_week_start = prev_week_start + datetime.timedelta(days = 1)
        
    
    return roll_avg_arr, negative_sent_arr, positive_sent_arr

In [96]:
data = rolling_avg_collector('AAPL')